# Introducción a SQLAlchemy

Todo este tutorial fue tomado de https://towardsdatascience.com/sqlalchemy-python-tutorial-79a577141a91 escrito por Vinay Kudari y algunos conceptos tomados de otros blogs.

# Antes de empezar
* Es una charla introductoria
* Estará enfocado en las bases de SQLAlchemy Core

### Suposiciones 

* Tienes conocimientos básicos de Python
* Tienes conocimientos básicos de SQL
* Tienes un entorno de Python instalado

# Introducción

- Estamos rodeados de datos 
- Necesitamos acceso a esos datos de manera rápida y fácil, no importa si desarrollas web, móvil o desktop
- Las base de datos relacionales son las mas comunes

## ¿Qué es SQL?

- SQL es un lenguaje estándar para almacenar, manipular y recuperar datos en bases de datos. Significa "Structured query language" (SQL)
- Se puede volver complicado integrar con el código de nuestras aplicaciones, ya que requeriria del manejo de strings (RAW code) y no es la solución mas segura.

## ¿Qué es SQLAlchemy?

- Es una librería de Python creada por Mike Bayer in 2005, que sin importar el dialecto, conecta la programación tradicional y las BD.
- Es considerada la herramienta por defecto para trabajar con Bases de datos en Python.
- Nos permite crear nuestros modelos de datos y consultas de una manera que se siente igual a clases y sentencias de Python
- Bases de datos soportadas por SQLAlchemy
- Hay dos modos de uso SQL Expression Language (conocido como SQLAlchemy Core) y ORM (Object-relational mapping)

### SQLAlchemy Core vs SQLAlchemy ORM

- Core es una abstracción del lenguaje SQL, esta enfocado en el esquema actual de la BD (tablas, filas, columnas, indices, etc)
- ORM esta mas enfocado en el modelo de la aplicación manteniendo el estado de los objetos  (lógica de negocios)
- Se pueden usar ambos al mismo tiempo ya que el ORM esta basado en SQLAlchemy Core

## Instalación

* Si usas anaconda o conda

```bash
conda install -c anaconda sqlalchemy 
```

* Si usas pip

```bash
pip install sqlalchemy
```

## Creando una base de datos

In [1]:
import sqlalchemy as db
import pandas as pd

In [66]:
engine = db.create_engine('sqlite:///test.sqlite') # Crea test.sqlite automaticamente
connection = engine.connect()
metadata = db.MetaData()

employee = db.Table('employee', metadata,
    db.Column('id', db.Integer()),
    db.Column('name', db.String(255), nullable=False),
    db.Column('salary', db.Float(), default=100.0),
    db.Column('active', db.Boolean(), default=True)
)

metadata.create_all(engine) # Crea la tabla

## Insertando datos

In [67]:
# Uno por uno
query = db.insert(employee).values(id=1, name='Maria', salary=6000.00, active=True)
ResultProxy = connection.execute(query)

In [68]:
# Insertando varios al mismo tiempo
query = db.insert(employee) 
values_list = [
    {'id':2, 'name':'Jose', 'salary':8000, 'active': False},
    {'id':3, 'name':'Carlos', 'salary':7000, 'active': True}
]
ResultProxy = connection.execute(query, values_list)

In [69]:
results = connection.execute(db.select([employee])).fetchall()
df = pd.DataFrame(results)
# Obtenemos las columnas del primer resultado
df.columns = results[0].keys()
df.head()

,id,name,salary,active
0,1,Maria,6000.0,True
1,2,Jose,8000.0,False
2,3,Carlos,7000.0,True


## Conectando a una base de datos

SQLAlchemy se puede usar para cargar tablas automáticamente desde una base de datos usando algo llamado reflexión. La reflexión es el proceso de leer la base de datos y construir los metadatos basados en esa información.

In [70]:
import sqlalchemy as db

In [74]:
engine = db.create_engine('sqlite:///census.sqlite')
connection = engine.connect()
metadata = db.MetaData()
census = db.Table('census', metadata, autoload=True, autoload_with=engine)

Aquí puedes encontrar ejemplos de distintas configuraciones: https://docs.sqlalchemy.org/en/latest/core/engines.html#postgresql

In [75]:
# Mostrando las columnas
print(census.columns.keys())

['state', 'sex', 'age', 'pop2000', 'pop2008']


In [76]:
# Mostrando toda la metadata de la tabla
print(repr(metadata.tables['census']))

Table('census', MetaData(bind=None), Column('state', VARCHAR(length=30), table=<census>), Column('sex', VARCHAR(length=1), table=<census>), Column('age', INTEGER(), table=<census>), Column('pop2000', INTEGER(), table=<census>), Column('pop2008', INTEGER(), table=<census>), schema=None)


## Consultando

In [82]:
# Equivalente a 'SELECT * FROM census'
query = db.select([census])

In [83]:
ResultProxy = connection.execute(query)

ResultProxy: el objeto devuelto por el método `.execute()`. Puede usarse de varias maneras para obtener los datos devueltos por la consulta.

In [84]:
ResultSet = ResultProxy.fetchall()

ResultSet: los datos reales solicitados en la consulta cuando se utiliza un método de búsqueda como `.fetchall()` en un ResultProxy.

In [85]:
ResultSet[:5]

[('Illinois', 'M', 0, 89600, 95012),
 ('Illinois', 'M', 1, 88445, 91829),
 ('Illinois', 'M', 2, 88729, 89547),
 ('Illinois', 'M', 3, 88868, 90037),
 ('Illinois', 'M', 4, 91947, 91111)]

In [86]:
# Convirtiendo a df
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df.head()

,state,sex,age,pop2000,pop2008
0,Illinois,M,0,89600,95012
1,Illinois,M,1,88445,91829
2,Illinois,M,2,88729,89547
3,Illinois,M,3,88868,90037
4,Illinois,M,4,91947,91111


### Filtrando

#### where

In [87]:
# SELECT * FROM census WHERE sex = F
query = db.select([census]).where(census.columns.sex == 'F')
result = connection.execute(query).fetchall()
result[:3]

[('Illinois', 'F', 0, 85910, 90286),
 ('Illinois', 'F', 1, 84396, 88126),
 ('Illinois', 'F', 2, 84764, 86291)]

#### in

In [89]:
# SELECT state, sex FROM census WHERE state IN (Texas, New York)
query = db.select(
    [census.columns.state, census.columns.sex]
).where(census.columns.state.in_(['Texas', 'New York']))

result = connection.execute(query).fetchall()
result

[('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),
 ('New York', 'M'),


#### and, or, not

In [91]:
# SELECT * FROM census WHERE state = 'California' AND NOT sex = 'M'
query = db.select([census]).where(
    db.and_(
        census.columns.state == 'California',
        census.columns.sex != 'M'
    )
)

result = connection.execute(query).fetchall()
result

[('California', 'F', 0, 239605, 274356),
 ('California', 'F', 1, 236543, 269140),
 ('California', 'F', 2, 240010, 262556),
 ('California', 'F', 3, 245739, 259061),
 ('California', 'F', 4, 254522, 255544),
 ('California', 'F', 5, 260264, 253899),
 ('California', 'F', 6, 261296, 247677),
 ('California', 'F', 7, 264083, 250117),
 ('California', 'F', 8, 270447, 233293),
 ('California', 'F', 9, 271482, 231261),
 ('California', 'F', 10, 270567, 235225),
 ('California', 'F', 11, 256656, 240906),
 ('California', 'F', 12, 249887, 249456),
 ('California', 'F', 13, 242724, 255296),
 ('California', 'F', 14, 240752, 256673),
 ('California', 'F', 15, 240170, 259496),
 ('California', 'F', 16, 233186, 266794),
 ('California', 'F', 17, 235767, 269445),
 ('California', 'F', 18, 234949, 285759),
 ('California', 'F', 19, 233477, 264961),
 ('California', 'F', 20, 233532, 257736),
 ('California', 'F', 21, 223990, 254077),
 ('California', 'F', 22, 222035, 258806),
 ('California', 'F', 23, 227742, 264302),
 (

#### order by

In [93]:
# SELECT * FROM census ORDER BY State DESC, pop2000
query = db.select([census]).order_by(
    db.desc(census.columns.state), census.columns.pop2000)
result = connection.execute(query).fetchall()
result[:3]

[('Wyoming', 'M', 84, 490, 643),
 ('Wyoming', 'M', 83, 515, 726),
 ('Wyoming', 'M', 82, 634, 792)]

#### Funciones

In [94]:
# SELECT SUM(pop2008) FROM census
query = db.select([db.func.sum(census.columns.pop2008)])
result = connection.execute(query).scalar()
result

302876613

#### group by

In [95]:
# SELECT SUM(pop2008) as pop2008, sex FROM census
query = db.select([
    db.func.sum(census.columns.pop2008).label('pop2008'),
    census.columns.sex
]).group_by(census.columns.sex)

result = connection.execute(query).fetchall()
result[:3]

[(153959198, 'F'), (148917415, 'M')]

#### distinct

In [96]:
# SELECT DISTINCT state FROM census
query = db.select([census.columns.state.distinct()])

result = connection.execute(query).fetchall()
result

[('Illinois',),
 ('New Jersey',),
 ('District of Columbia',),
 ('North Dakota',),
 ('Florida',),
 ('Maryland',),
 ('Idaho',),
 ('Massachusetts',),
 ('Oregon',),
 ('Nevada',),
 ('Michigan',),
 ('Wisconsin',),
 ('Missouri',),
 ('Washington',),
 ('North Carolina',),
 ('Arizona',),
 ('Arkansas',),
 ('Colorado',),
 ('Indiana',),
 ('Pennsylvania',),
 ('Hawaii',),
 ('Kansas',),
 ('Louisiana',),
 ('Alabama',),
 ('Minnesota',),
 ('South Dakota',),
 ('New York',),
 ('California',),
 ('Connecticut',),
 ('Ohio',),
 ('Rhode Island',),
 ('Georgia',),
 ('South Carolina',),
 ('Alaska',),
 ('Delaware',),
 ('Tennessee',),
 ('Vermont',),
 ('Montana',),
 ('Kentucky',),
 ('Utah',),
 ('Nebraska',),
 ('West Virginia',),
 ('Iowa',),
 ('Wyoming',),
 ('Maine',),
 ('New Hampshire',),
 ('Mississippi',),
 ('Oklahoma',),
 ('New Mexico',),
 ('Virginia',),
 ('Texas',)]

### Joins

Si tiene dos tablas que ya tienen una relación establecida, puede usar esa relación automáticamente simplemente agregando las columnas que queremos de cada tabla a la declaración de selección.

`select([census.columns.pop2008, state_fact.columns.abbreviation])`

In [97]:
import sqlalchemy as db
import pandas as pd

engine = db.create_engine('sqlite:///census.sqlite')
connection = engine.connect()
metadata = db.MetaData()

census = db.Table('census', metadata, autoload=True, autoload_with=engine)
state_fact = db.Table('state_fact', metadata, autoload=True, autoload_with=engine)

### Automatic Joins

In [98]:
query = db.select([census.columns.pop2008, state_fact.columns.abbreviation])
result = connection.execute(query).fetchall()
df = pd.DataFrame(result)
df.columns = result[0].keys()
df.head(5)

,pop2008,abbreviation
0,95012,IL
1,95012,NJ
2,95012,ND
3,95012,OR
4,95012,DC


### Manual Join

In [99]:
query = db.select([census, state_fact])
query = query.select_from(
    census.join(state_fact, census.columns.state == state_fact.columns.name))
result = connection.execute(query).fetchall()

df = pd.DataFrame(result)
df.columns = result[0].keys()
df.head(5)

,state,sex,age,pop2000,pop2008,id,name,abbreviation,country,type,...,occupied,notes,fips_state,assoc_press,standard_federal_region,census_region,census_region_name,census_division,census_division_name,circuit_court
0,Illinois,M,0,89600,95012,13,Illinois,IL,USA,state,...,occupied,,17,Ill.,V,2,Midwest,3,East North Central,7
1,Illinois,M,1,88445,91829,13,Illinois,IL,USA,state,...,occupied,,17,Ill.,V,2,Midwest,3,East North Central,7
2,Illinois,M,2,88729,89547,13,Illinois,IL,USA,state,...,occupied,,17,Ill.,V,2,Midwest,3,East North Central,7
3,Illinois,M,3,88868,90037,13,Illinois,IL,USA,state,...,occupied,,17,Ill.,V,2,Midwest,3,East North Central,7
4,Illinois,M,4,91947,91111,13,Illinois,IL,USA,state,...,occupied,,17,Ill.,V,2,Midwest,3,East North Central,7


## Actualizando registros

In [100]:
import sqlalchemy as db
import pandas as pd

In [101]:
engine = db.create_engine('sqlite:///test.sqlite')
metadata = db.MetaData()
connection = engine.connect()
employee = db.Table('employee', metadata, autoload=True, autoload_with=engine)

In [102]:
results = connection.execute(db.select([employee])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(4)

,id,name,salary,active
0,1,Maria,6000.0,True
1,2,Jose,8000.0,False
2,3,Carlos,7000.0,True


In [103]:
# Build a statement to update the salary to 10000
query = db.update(employee).values(salary = 10000)
query = query.where(employee.columns.id == 1)
results = connection.execute(query)

In [104]:
results = connection.execute(db.select([employee])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(4)

,id,name,salary,active
0,1,Maria,10000.0,True
1,2,Jose,8000.0,False
2,3,Carlos,7000.0,True


## Eliminando registros

In [105]:
import sqlalchemy as db
import pandas as pd

In [106]:
engine = db.create_engine('sqlite:///test.sqlite')
metadata = db.MetaData()
connection = engine.connect()
employee = db.Table('employee', metadata, autoload=True, autoload_with=engine)

In [107]:
results = connection.execute(db.select([emp])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(4)

,id,name,salary,active
0,1,Maria,10000.0,True
1,2,Jose,8000.0,False
2,3,Carlos,7000.0,True


In [108]:
# Build a statement to delete where salary < 100000
query = db.delete(employee)
query = query.where(employee.columns.salary == 10000)
results = connection.execute(query)

In [109]:
results = connection.execute(db.select([emp])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(4)

,id,name,salary,active
0,2,Jose,8000.0,False
1,3,Carlos,7000.0,True


## Eliminando todo

```
table_name.drop(engine) #drops a single table
metadata.drop_all(engine) #drops all the tables in the database
```

# Recursos

https://www.sqlalchemy.org/

https://www.blog.pythonlibrary.org/2010/02/03/another-step-by-step-sqlalchemy-tutorial-part-1-of-2/

https://towardsdatascience.com/sqlalchemy-python-tutorial-79a577141a91

https://overiq.com/sqlalchemy-101/crud-using-sqlalchemy-core/